This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    Polymers
    packmol

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.packmol import PackmolRunner
from polymers.utils import get_topology

Set the start, bulk, and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
s_charges = [-0.1187, 0.0861, 0.0861, 0.0861, 
             -0.2792, -0.0326, 0.0861, 0.0861]
peo_start.add_site_property("charge", s_charges)
#ff_map = ["C", "H", "H", "H", "O", "C", "H", "H"]
#peo_start.add_site_property("ff_map", ff_map)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
b_charges = [-0.0326, 0.0861, 0.0861, -0.2792, 
             -0.0326, 0.0861, 0.0861]
peo_bulk.add_site_property("charge", b_charges)
#ff_map = ["C", "H", "H", "O", "C", "H", "H"]
#peo_bulk.add_site_property("ff_map", ff_map)
head = 0
tail = 4

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
e_charges = [-0.0326, 0.0861, 0.0861, -0.2792, 
             -0.1187, 0.0861, 0.0861, 0.0861]
peo_end.add_site_property("charge", e_charges)
#ff_map = ["C", "H", "H", "O", "C", "H", "H", "H"]
#peo_end.add_site_property("ff_map", ff_map)
e_head = 0
e_tail = 4

Create polymer via random walk

In [3]:
n_units = 58
link_distance = 1.5075

# create the polymer
peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      peo_bulk, head, tail, 
                      peo_end, e_head, e_tail, 
                      n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk, head, tail, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

peo_polymer.molecule.to(filename="polymer.xyz", fmt="xyz")
#peo_polymer.molecule.get_covalent_bonds(tol=0.1)

Topology of the peo_polymer, from the linear chain

In [4]:
from pymatgen.io.lammps.topology import Topology
topology = Topology.from_molecule(peo_polymer_linear.molecule)
print len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)
print topology.bonds

408 407 754 861
[[0, 1, ('C', 'H')], [0, 2, ('C', 'H')], [0, 3, ('C', 'H')], [0, 4, ('C', 'O')], [4, 5, ('O', 'C')], [5, 6, ('C', 'H')], [5, 7, ('C', 'H')], [5, 8, ('C', 'C')], [8, 9, ('C', 'H')], [8, 10, ('C', 'H')], [8, 11, ('C', 'O')], [11, 12, ('O', 'C')], [12, 13, ('C', 'H')], [12, 14, ('C', 'H')], [12, 15, ('C', 'C')], [15, 16, ('C', 'H')], [15, 17, ('C', 'H')], [15, 18, ('C', 'O')], [18, 19, ('O', 'C')], [19, 20, ('C', 'H')], [19, 21, ('C', 'H')], [19, 22, ('C', 'C')], [22, 23, ('C', 'H')], [22, 24, ('C', 'H')], [22, 25, ('C', 'O')], [25, 26, ('O', 'C')], [26, 27, ('C', 'H')], [26, 28, ('C', 'H')], [26, 29, ('C', 'C')], [29, 30, ('C', 'H')], [29, 31, ('C', 'H')], [29, 32, ('C', 'O')], [32, 33, ('O', 'C')], [33, 34, ('C', 'H')], [33, 35, ('C', 'H')], [33, 36, ('C', 'C')], [36, 37, ('C', 'H')], [36, 38, ('C', 'H')], [36, 39, ('C', 'O')], [39, 40, ('O', 'C')], [40, 41, ('C', 'H')], [40, 42, ('C', 'H')], [40, 43, ('C', 'C')], [43, 44, ('C', 'H')], [43, 45, ('C', 'H')], [43, 46, ('C'

Use Packmol to pack the polymer chains into a box

In [5]:
# one polymer molecule is fixed at the origin, surrounded by 5 polymer chains within a radius of 200A
packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""}]
                  #{"number": 5, "inside sphere": [0, 0, 0, 100.0]}]
pmr = PackmolRunner([peo_polymer.molecule],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

packed molecule written to poly_packed.xyz


Read in the force field parameters

In [12]:
from pymatgen.io.lammps.force_field import ForceField
forcefield = ForceField.from_file("ff_data.yaml")

Generate lammps data file from the topology and the forcefield

In [7]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols = [peo_polymer.molecule]
mols_number = [1]
box_size = [[0.0, 300.0], [0.0, 300.0], [0.0, 300.0]]
molecule = packed_polymer
topologies = [topology]
lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(mols, mols_number, 
                                                                   box_size, molecule, 
                                                                   forcefield, topologies)

Write the data file(uncomment the write line)

In [9]:
print str(lammps_ff_data)

#write the data file
lammps_ff_data.write_data_file("lammps_data.dat")

Data file generated by pymatgen

408 atoms
407 bonds
754 angles
861 dihedrals

3 atom types
3 bond types
5 angle types
5 dihedral types

0.0 300.0 xlo xhi
0.0 300.0 ylo yhi
0.0 300.0 zlo zhi

Masses 

1 1.00794
2 12.0107
3 15.9994

Pair Coeffs 

1 75844.8 0.2461 396.9
2 33702.4 0.2796 503.0
3 2649.6 0.2674 27.22
4 4320.0 0.2928 137.6
5 14976.0 0.3236 637.6

Bond Coeffs 

1 1000 1.4115
2 1000 1.1041
3 1000 1.5075

Angle Coeffs 

1 42.9 110.1
2 38.5 109.47
3 56.0 109.48
4 86.0 108.54
5 74.5 108.05

Dihedral Coeffs 

1 0.0 0.0 -0.73 0.0
2 0.0 0.0 0.28 0.0
3 1.76 0.67 0.04 0.0
4 0.0 0.0 0.28 0.0
5 0.41 -2.1 -0.6 -0.82

Atoms 

1 1 1 -0.1187 111.299952514 166.023390197 159.456917597
2 1 1 0.0861 111.284492514 164.497290197 158.555267597
3 1 1 0.0861 111.284492514 164.497290197 160.358567597
4 1 1 0.0861 113.229342514 163.155260197 158.555267597
5 1 1 -0.2792 113.229342514 163.155260197 160.358567597
6 1 1 -0.0326 112.497502514 163.660270197 157.961907597
7 1 1 0.0861 113.981742514 162.63608

Atoms 408 and 1 should not be bonded